# Transform Face Dataset to Cropped Face dataset

The goal of the Cropped dataset is to see if elimination of background and other noise in an image results in better classification performance.
It is created by using a [Multi-task CNN](https://github.com/ipazc/mtcnn) face detector to detect face in an image and saving only the face as a new image.

My Cropped dataset consists of the same classes and with the same number of images, contrary to HOG and SVM approach in the [paper](https://lib.jucs.org/article/104490/) "Transfer Learning with EfficientNetV2S for Automatic Face Shape Classification" 

In [36]:
import glob
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
        
import imutils

from mtcnn import MTCNN

In [ ]:
!pip install roboflow

# !!!! change this
API_KEY = 'Your api key'

from roboflow import Roboflow
rf = Roboflow(api_key=API_KEY)
project = rf.workspace("shapeofyou").project("shapeofyou")
version = project.version(2)
dataset = version.download("folder")


In [ ]:
def crop_ds(ds_name, sub_ds, class_name, image_format='jpg'):

    detector = MTCNN()
    min_conf = 0.9  
    offset = 20

    os.makedirs('cropped_ds/'+sub_ds, exist_ok=True)
    os.makedirs('cropped_ds/'+sub_ds+'/'+class_name, exist_ok=True)

    # sometimes resources are exhausted and this script should be run multiple times
    # to assure that no image is twice handled
    # sets of files in both folders are compared
    # only not transformed images are taken into account
    image_list = glob.glob(ds_name+"/"+sub_ds+"/"+class_name+"/*."+image_format)
    new_image_list = ['/'.join(x.split('/')[1:]) for x in image_list]
    cropped_list = glob.glob('cropped_ds'+"/"+sub_ds+"/"+class_name+"/*."+image_format)
    new_cropped_list = ['/'.join(x.split('/')[1:]) for x in cropped_list]
    left = set(new_image_list) - set(new_cropped_list)
    left_list = [ds_name+"/"+x for x in left]
    print(len(left_list))
    for image_path in left_list:
        new_im_path = os.path.join('cropped_ds', sub_ds, '/'.join(image_path.split('/')[-2:]))

        # in case of error you know what image caused it
        print(new_im_path)
        img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        #  in case of very big image
        # import imutils
        # img = imutils.resize(img, width=1280)
        h,w,ch = img.shape 
        area = 0
        final_face = None
        detections = detector.detect_faces(img)

        # transform only face with the biggest area 
        for det in detections:
            if det['confidence'] >= min_conf:
                x, y, width, height = det['box']
                object = img[max(y-offset,0):min(y+height+offset,h), max(0,x-offset):min(w,x+width+offset), :]
                object_area = object.shape[0]*object.shape[1]
                if (object_area > area):
                    area = object_area
                    final_face = object
        cv2.imwrite(new_im_path, cv2.cvtColor(final_face, cv2.COLOR_RGB2BGR))               

It takes time and memory, so each transformation has its own cell. Test images are all in one cell, because their number is not too big.

In [ ]:
crop_ds('shapeofyou-2', 'train', 'Heart')

In [ ]:
crop_ds('shapeofyou-2', 'train', 'Oblong')

In [ ]:
crop_ds('shapeofyou-2', 'train', 'Oval')

In [ ]:
crop_ds('shapeofyou-2', 'train', 'Round')

In [ ]:
crop_ds('shapeofyou-2', 'train', 'Square')

In [ ]:
crop_ds('shapeofyou-2', 'test', 'Heart')
crop_ds('shapeofyou-2', 'test', 'Oblong')
crop_ds('shapeofyou-2', 'test', 'Oval')
crop_ds('shapeofyou-2', 'test', 'Round')
crop_ds('shapeofyou-2', 'test', 'Square')

To test one image if something goes wrong

In [ ]:
# show image with detections
image_path = 'shapeofyou-2/train/Oval/oval-735-_jpg.rf.250a94bb29ff9f51398cf6f63f98dfc5.jpg'
img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
detector = MTCNN()
detections = detector.detect_faces(img)
detections
img_with_dets = img.copy()
min_conf = 0.9
for det in detections:
    if det['confidence'] >= min_conf:
        x, y, width, height = det['box']
        keypoints = det['keypoints']
        cv2.rectangle(img_with_dets, (x-20,y-20), (x+width+20,y+height+20), (0,155,255), 2)
        cv2.circle(img_with_dets, (keypoints['left_eye']), 2, (0,155,255), 2)
        cv2.circle(img_with_dets, (keypoints['right_eye']), 2, (0,155,255), 2)
        cv2.circle(img_with_dets, (keypoints['nose']), 2, (0,155,255), 2)
        cv2.circle(img_with_dets, (keypoints['mouth_left']), 2, (0,155,255), 2)
        cv2.circle(img_with_dets, (keypoints['mouth_right']), 2, (0,155,255), 2)
plt.figure(figsize = (10,10))
plt.imshow(img_with_dets)
plt.axis('off')

In [ ]:
# show cropped face
offset = 20
h,w,ch = img_with_dets.shape
area = 0
final_face = None
for det in detections:
    if det['confidence'] >= min_conf:
        x, y, width, height = det['box']
        object = img[max(y-offset,0):min(y+height+offset,h), max(0,x-offset):min(w,x+width+offset), :]
        object_area = object.shape[0]*object.shape[1]
        if (object_area > area):
            area = object_area
            final_face = object.copy()
new_im_path = os.path.join('cropped_ds', 'train', '/'.join(image_path.split('/')[-2:]))
plt.figure(figsize = (10,10))
plt.imshow(final_face)
plt.axis('off')

## Data augmentation

The augmentation method increases the number of images in the dataset while making it more difficult for the network to learn, because none of the images are completely standard. In the [paper](https://lib.jucs.org/article/104490/), the image variants include:
- image rotation for a value between -50◦ and 30◦
- adding Gaussian noise to an image
- horizontal image mirroring
- changing the contrast of an image for gamma value of 2.

In [59]:
def augment_ds(ds_name, split_name, class_name, image_format='jpg'):

    image_list = glob.glob(ds_name+"/"+ split_name + '/' +class_name+"/*."+image_format)
    print(len(image_list))
    for image_path in image_list:
        new_im_path = image_path[:-4]

        img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        
        # flip image
        flipped = cv2.flip(img, 1)
        cv2.imwrite(new_im_path+'flipped.jpg', cv2.cvtColor(flipped, cv2.COLOR_RGB2BGR))  
        
        # rotate image
        angle = np.random.randint(-50, 30)        
        rotated = imutils.rotate(img, angle)
        cv2.imwrite(new_im_path+'rotated.jpg', cv2.cvtColor(rotated, cv2.COLOR_RGB2BGR))  

        # add Gaussian noise
        mean = 0
        std=1
        noise = np.random.normal(mean, std, img.shape).astype(np.uint8)
        noisy_image = cv2.add(img, noise)
        cv2.imwrite(new_im_path+'gauss.jpg', cv2.cvtColor(noisy_image, cv2.COLOR_RGB2BGR))
    
        gamma = 0.8
        invGamma = 1.0 / gamma
        table = np.array([((i / 255.0) ** invGamma) * 255
            for i in np.arange(0, 256)]).astype("uint8")
        # apply gamma correction using the lookup table
        contrasted = cv2.LUT(img, table)
        cv2.imwrite(new_im_path+'contrast.jpg', cv2.cvtColor(contrasted, cv2.COLOR_RGB2BGR))

        # plt.figure(figsize = (10,10))
        # plt.imshow(img)
        # plt.axis('off')

In [60]:
augment_ds('augmented_ds', 'train', 'Heart')
augment_ds('augmented_ds', 'train', 'Oblong')
augment_ds('augmented_ds', 'train', 'Oval')
augment_ds('augmented_ds', 'train', 'Round')
augment_ds('augmented_ds', 'train', 'Square')

989


In [97]:
from sklearn.model_selection import train_test_split

def split_to_train_test(ds_name, split_name, class_name, image_format='jpg'):
   
   image_list = glob.glob(ds_name + "/" + split_name + '/' +class_name+"/*."+image_format)
   x_train ,x_test = train_test_split(image_list,test_size=0.2)
   os.makedirs(ds_name+'/test', exist_ok=True)
   os.makedirs(ds_name+'/test/'+ class_name, exist_ok=True)
   for im in x_test:
      splitted = im.split('/')
      splitted.insert(1, 'test')
      new_path = '/'.join(splitted)
      os.rename(im, new_path)
   

In [96]:
split_to_train_test('augmented_ds', 'train', 'Heart')
split_to_train_test('augmented_ds', 'train', 'Oblong')
split_to_train_test('augmented_ds', 'train', 'Oval')
split_to_train_test('augmented_ds', 'train', 'Round')
split_to_train_test('augmented_ds', 'train', 'Square')

4905
3924
981
['augmented_ds/Heart/heart-465-_jpg.rf.24396be8a6e404abf850bed684374556.jpg', 'augmented_ds/Heart/heart-604-_jpg.rf.12802d4124e65cb7253b09b5c52d08c5gauss.jpg', 'augmented_ds/Heart/heart-719-_jpg.rf.522c8226b2894398fd310507def2ea86gauss.jpg', 'augmented_ds/Heart/heart-672-_jpg.rf.1489b85304ce733984bc0ff2f8de5706rotated.jpg']
augmented_ds/test/Heart/heart-465-_jpg.rf.24396be8a6e404abf850bed684374556.jpg
augmented_ds/test/Heart/heart-604-_jpg.rf.12802d4124e65cb7253b09b5c52d08c5gauss.jpg
augmented_ds/test/Heart/heart-719-_jpg.rf.522c8226b2894398fd310507def2ea86gauss.jpg
augmented_ds/test/Heart/heart-672-_jpg.rf.1489b85304ce733984bc0ff2f8de5706rotated.jpg
augmented_ds/test/Heart/heart-234-_jpg.rf.348859974ab23cfcad7a2334916d3ab1.jpg
augmented_ds/test/Heart/heart-796-_jpg.rf.d06ec2fb32b0092769d2d00fa2d65c9bgauss.jpg
augmented_ds/test/Heart/heart-272-_jpg.rf.ed6ce156637ebb5d992283fd86ba7a04flipped.jpg
augmented_ds/test/Heart/heart-351-_jpg.rf.b04fff69b2e7f87d4b43437a38cf2b2c.jpg